## <span style="color:green; text-aling:justify;">Representação de texto: embeddings e modelos como Word2Vec</span>

### 1 - Preparação do Ambiente

In [ ]:
# Importações básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.manifold import TSNE
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

# Download de recursos NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

### 2 - Preparando um Dataset para Treinamento

#### 2.1 - Carregando Dados de Exemplo

In [ ]:
# Dados de exemplo - críticas de filmes (simplificadas)
textos = [
    "Este filme é incrível, adorei a atuação do protagonista",
    "A direção de fotografia é espetacular e o roteiro é envolvente",
    "Péssimo filme, desperdicei meu tempo assistindo isso",
    "Os atores são talentosos mas o roteiro é fraco",
    "Cinematografia belíssima, recomendo assistir no cinema",
    "Não gostei da história, personagens mal desenvolvidos",
    "A trilha sonora combina perfeitamente com as cenas",
    "Filme entediante, previsível do início ao fim",
    "Os efeitos especiais são impressionantes, tecnologia de ponta",
    "História emocionante, chorei no final do filme"
]

# Verificando os dados
for i, texto in enumerate(textos[:3]):  # Mostrando apenas os 3 primeiros
    print(f"Texto {i+1}: {texto}")

#### 2.2 - Pré-processamento do Texto

In [ ]:
from nltk.corpus import stopwords

def preprocessar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()

    # Remover caracteres especiais e números
    texto = re.sub(r'[^a-záàâãéèêíïóôõöúçñ ]', '', texto)

    # Tokenizar
    tokens = word_tokenize(texto)

    # Remover stopwords (opcional, dependendo da aplicação)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

# Aplicar pré-processamento a todos os textos
textos_preprocessados = [preprocessar_texto(texto) for texto in textos]

# Verificar resultado
print("Exemplo de texto original:")
print(textos[0])
print("\nDepois do pré-processamento:")
print(textos_preprocessados[0])

### 3 - Implementando Word2Vec com Gensim

#### 3.1 -  Treinando um Modelo do Zero

In [ ]:
# Definir parâmetros do modelo
vector_size = 100    # Dimensionalidade dos vetores
window = 5           # Tamanho da janela de contexto
min_count = 1        # Frequência mínima das palavras
workers = 4          # Número de threads para treinamento
sg = 1               # Modelo Skip-gram (1) ou CBOW (0)

# Treinar o modelo
model = Word2Vec(
    sentences=textos_preprocessados,
    vector_size=vector_size,
    window=window,
    min_count=min_count,
    workers=workers,
    sg=sg
)

print(f"Modelo treinado com {len(model.wv.key_to_index)} palavras no vocabulário")


#### 3.2 - Explorando o Modelo Treinado

In [ ]:
# Listar algumas palavras do vocabulário
palavras = list(model.wv.key_to_index.keys())
print("Algumas palavras do vocabulário:")
print(palavras[:10])  # Primeiras 10 palavras

# Verificar o vetor de uma palavra específica
if 'filme' in model.wv:
    vetor_filme = model.wv['filme']
    print(f"\nVetor da palavra 'filme' (primeiras 10 dimensões):")
    print(vetor_filme[:10])
    print(f"Dimensionalidade do vetor: {len(vetor_filme)}")


#### 3.3 - Palavras Mais Similares

In [ ]:
# Encontrar palavras mais similares a 'filme'
if 'filme' in model.wv:
    similares = model.wv.most_similar('filme', topn=5)
    print("\nPalavras mais similares a 'filme':")
    for palavra, similaridade in similares:
        print(f"{palavra}: {similaridade:.4f}")


#### 3.4 - Salvando e Carregando o Modelo

In [ ]:
# Salvar o modelo
model.save("word2vec_filmes.model")

# Carregando o modelo salvo
modelo_carregado = Word2Vec.load("word2vec_filmes.model")
print("Modelo carregado com sucesso!")

### 4 - Usando Modelos Pré-treinados

In [ ]:
# Listar modelos disponíveis
print("Alguns modelos disponíveis no gensim-data:")
print([nome for nome in list(api.info()['models'].keys())[:10]])

# Carregar um modelo pré-treinado (word2vec-google-news-300)
# Nota: Isso pode demorar um pouco na primeira execução (download)
try:
    # Para português, você pode tentar 'word2vec-portuguese' se disponível
    # ou usar modelos do NILC: http://nilc.icmc.usp.br/embeddings
    modelo_pretreino = api.load("glove-wiki-gigaword-100")  # Mais rápido que word2vec-google-news-300
    print(f"Modelo pré-treinado carregado com {len(modelo_pretreino.key_to_index)} palavras")

    # Verificar palavras similares usando modelo pré-treinado
    if 'computer' in modelo_pretreino:
        similares = modelo_pretreino.most_similar('computer', topn=5)
        print("\nPalavras mais similares a 'computer':")
        for palavra, similaridade in similares:
            print(f"{palavra}: {similaridade:.4f}")
except Exception as e:
    print(f"Erro ao carregar modelo pré-treinado: {e}")
    print("Tente outro modelo ou prossiga sem esta parte")


### 5 - Operações Vetoriais e Analogias

In [ ]:
# Usando modelo pré-treinado para analogias (se disponível)
try:
    # Famosa analogia: rei - homem + mulher = rainha
    if all(word in modelo_pretreino for word in ['king', 'man', 'woman']):
        resultado = modelo_pretreino.most_similar(
            positive=['king', 'woman'],
            negative=['man'],
            topn=3
        )
        print("\nAnalogia: rei - homem + mulher =")
        for palavra, similaridade in resultado:
            print(f"{palavra}: {similaridade:.4f}")
except:
    print("Não foi possível realizar a operação de analogia com o modelo disponível")

# Podemos tentar outras analogias com nosso modelo treinado
# Por exemplo: bom - positivo + negativo = ruim
# (Dependendo do tamanho do corpus, pode não funcionar bem para modelos pequenos)


### 6 -  Calculando Similaridade entre Palavras

In [ ]:
# Função para calcular similaridade entre pares de palavras
def calcular_similaridade(modelo, pares_palavras):
    resultados = []
    for par in pares_palavras:
        palavra1, palavra2 = par
        if palavra1 in modelo.wv and palavra2 in modelo.wv:
            similaridade = modelo.wv.similarity(palavra1, palavra2)
            resultados.append((par, similaridade))
        else:
            resultados.append((par, "Uma ou ambas as palavras não estão no vocabulário"))
    return resultados

# Pares de palavras para testar
pares = [
    ('filme', 'cinema'),
    ('bom', 'ruim'),
    ('ator', 'atuação'),
    ('filme', 'protagonista')
]

# Calcular similaridades
similaridades = calcular_similaridade(model, pares)

# Exibir resultados
print("\nSimilaridade entre pares de palavras:")
for (palavra1, palavra2), similaridade in similaridades:
    if isinstance(similaridade, float):
        print(f"{palavra1} - {palavra2}: {similaridade:.4f}")
    else:
        print(f"{palavra1} - {palavra2}: {similaridade}")


### 7 - Visualização de Word Embeddings

In [ ]:
def visualizar_embeddings(modelo, palavras=None, n_palavras=50):
    """
    Visualiza os embeddings em um espaço 2D usando t-SNE.
    Args:
        modelo: Modelo Word2Vec
        palavras: Lista de palavras específicas para visualizar (opcional)
        n_palavras: Número de palavras mais frequentes a visualizar (se palavras=None)
    """
    # Obter palavras e vetores
    if palavras is None:
        palavras = list(modelo.wv.key_to_index.keys())[:n_palavras]
    else:
        # Filtrar palavras que não estão no vocabulário
        palavras = [p for p in palavras if p in modelo.wv]
    
    if not palavras:
        print("Nenhuma palavra válida para visualização")
        return
    
    # Obter vetores e converter para array NumPy
    vetores = np.array([modelo.wv[palavra] for palavra in palavras])
    
    # Reduzir dimensionalidade com t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(palavras)-1))
    vetores_2d = tsne.fit_transform(vetores)
    
    # Criar gráfico
    plt.figure(figsize=(12, 10))
    
    # Plotar pontos
    x = vetores_2d[:, 0]
    y = vetores_2d[:, 1]
    plt.scatter(x, y, c='blue', alpha=0.7)
    
    # Adicionar rótulos (palavras)
    for i, palavra in enumerate(palavras):
        plt.annotate(
            palavra,
            xy=(x[i], y[i]),
            xytext=(5, 2),
            textcoords='offset points',
            fontsize=10
        )
    
    plt.title("Visualização t-SNE dos Word Embeddings")
    plt.grid(True)
    plt.show()

### 8 - Aplicação Prática: Classificação de Sentimentos com Embeddings

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Dados rotulados para exemplo
textos_rotulados = textos  # Usando os mesmos textos de antes
sentimentos = [1, 1, 0, 0, 1, 0, 1, 0, 1, 1]  # 1: positivo, 0: negativo

# Função para gerar vetores de documento usando embeddings
def texto_para_vetor(texto, modelo):
    """Converte um texto em um vetor médio dos embeddings de suas palavras"""
    palavras = preprocessar_texto(texto)
    # Filtrar palavras que estão no vocabulário do modelo
    palavras_no_vocab = [p for p in palavras if p in modelo.wv]
    if not palavras_no_vocab:
        # Se nenhuma palavra estiver no vocabulário, retorna vetor de zeros
        return np.zeros(modelo.vector_size)
    # Calcular a média dos vetores das palavras
    vetores = [modelo.wv[palavra] for palavra in palavras_no_vocab]
    return np.mean(vetores, axis=0)

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    textos_rotulados, sentimentos, test_size=0.3, random_state=42
)

# 1. Abordagem com TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

clf_tfidf = LogisticRegression(random_state=42)
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

# 2. Abordagem com Word Embeddings
X_train_emb = np.array([texto_para_vetor(texto, model) for texto in X_train])
X_test_emb = np.array([texto_para_vetor(texto, model) for texto in X_test])

clf_emb = LogisticRegression(random_state=42)
clf_emb.fit(X_train_emb, y_train)
y_pred_emb = clf_emb.predict(X_test_emb)

# Comparar resultados
print("\nResultados com TF-IDF:")
print(classification_report(y_test, y_pred_tfidf))

print("\nResultados com Word Embeddings:")
print(classification_report(y_test, y_pred_emb))


 ### 9 - Word Embeddings Contextuais com spaCy

In [ ]:
import spacy

# Carregar modelo spaCy (pequeno)
try:
    nlp = spacy.load("pt_core_news_sm")  # Para português
    # Alternativa para inglês: nlp = spacy.load("en_core_web_sm")

    # Exemplo de texto
    texto = "O banco está cheio de dinheiro. Eu sentei no banco da praça."

    # Processar o texto
    doc = nlp(texto)

    # Examinar embeddings para cada ocorrência da palavra "banco"
    for token in doc:
        if token.text.lower() == "banco":
            contexto = doc[max(0, token.i-3):min(len(doc), token.i+4)]
            print(f"Contexto: {contexto}")
            print(f"Vetor (10 primeiras dimensões): {token.vector[:10]}")
            print(f"Dimensão do vetor: {len(token.vector)}")
            print("-" * 50)
except Exception as e:
    print(f"Erro ao carregar spaCy: {e}")
    print("Talvez seja necessário instalar os modelos com:")
    print("python -m spacy download pt_core_news_sm")
    print("ou")
    print("python -m spacy download en_core_web_sm")


### 10 - FastText: Embeddings com Subpalavras

In [ ]:
from gensim.models import FastText

# Treinar modelo FastText
modelo_fasttext = FastText(
    sentences=textos_preprocessados,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1
)

# Testar com palavras que não estão no corpus
palavras_teste = ["filmagem", "cinematográfico", "atuações"]

print("\nEmbeddings para palavras fora do vocabulário (FastText):")
for palavra in palavras_teste:
    # Verificar se a palavra está no vocabulário original
    no_vocab = palavra in model.wv
    # Obter vetor do FastText (funciona mesmo para palavras fora do vocabulário)
    vetor = modelo_fasttext.wv[palavra]
    print(f"'{palavra}' (no vocabulário: {no_vocab})")
    print(f"Primeiras 5 dimensões do vetor: {vetor[:5]}")

    # Encontrar palavras similares
    similares = modelo_fasttext.wv.most_similar(palavra, topn=3)
    print(f"Palavras similares a '{palavra}':")
    for p, sim in similares:
        print(f"  {p}: {sim:.4f}")
    print()


## 11 - . Exercícios Práticos

In [ ]:
def similaridade_documentos(doc1, doc2, modelo):
    """Calcula a similaridade entre dois documentos usando embeddings"""
    vetores_doc1 = [texto_para_vetor(parte, modelo) for parte in doc1]
    vetores_doc2 = [texto_para_vetor(parte, modelo) for parte in doc2]

    similaridades = []
    for vetor1 in vetores_doc1:
        for vetor2 in vetores_doc2:
            similaridade = np.dot(vetor1, vetor2) / (np.linalg.norm(vetor1) * np.linalg.norm(vetor2))
            similaridades.append(similaridade)
    
    return np.mean(similaridades)

# Exercício: Calcule a similaridade entre os documentos abaixo
documento1 = "O filme tem uma história envolvente e atuações convincentes"
documento2 = "A narrativa do filme é cativante e os atores são excelentes"
documento3 = "O restaurante tem comida deliciosa e preços acessíveis"

# Calcular similaridades (implemente sua solução)
documentos = [
    documento1,
    documento2,
    documento3
]


docsProcessados = [preprocessar_texto(doc) for doc in documentos]

similaridadeDocs = [
    similaridade_documentos(doc1, doc2, spacy)  
    for doci, doc1 in enumerate(docsProcessados)  
    for docj, doc2 in enumerate(docsProcessados) if doci < docj
]

print(docsProcessados)


AttributeError: module 'spacy' has no attribute 'wv'